In [8]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
from pathlib import Path
from datetime import datetime as dt
import pandas as pd

In [52]:
path="C:\my lap\Graduation proj\ArabicNER\Training\Arabic-named-entity-recognition-master\ANERCorp\ANE1.xlsx"
xlsx = pd.ExcelFile(path)
df=pd.read_excel(xlsx, header=None)
df

,0,1,2
0,فرانكفورت,B-LOC,NaN
1,(د,O,NaN
2,ب,O,NaN
3,أ),O,NaN
4,أعلن,O,NaN
...,...,...,...
150402,التواضع,M,NaN
150403,رحابة الصدر,M,NaN
150404,حسن السياسة,M,NaN
150405,مرونة التفكير,M,NaN


In [53]:
df = df.drop(2, 1)
df = df.rename(columns={0: 'text', 1: 'label'})
df

,text,label
0,فرانكفورت,B-LOC
1,(د,O
2,ب,O
3,أ),O
4,أعلن,O
...,...,...
150402,التواضع,M
150403,رحابة الصدر,M
150404,حسن السياسة,M
150405,مرونة التفكير,M


In [54]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.2)
train_arr = []
test_arr = []
train_lbl = []
test_lbl = []

In [55]:
train_arr=train['text'].astype(str)
train_lbl=train['label'].astype(str)
test_arr=test['text'].astype(str)
test_lbl=test['label'].astype(str)

In [56]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(train_arr)
train_mat = vectorizer.transform(train_arr)
tfidf = TfidfTransformer()
tfidf.fit(train_mat)
train_tfmat = tfidf.transform(train_mat)
test_mat = vectorizer.transform(test_arr)
test_tfmat = tfidf.transform(test_mat)

In [57]:
del df
del test_arr
del train_arr

In [58]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.svm import LinearSVC
train_tfmat

lsvm=LinearSVC()
lsvm.fit(train_tfmat,train_lbl)

LinearSVC()

In [59]:
y_pred_lsvm=lsvm.predict(test_tfmat)

In [60]:
test=['ألمانيا']
test_str = vectorizer.transform(test)
test_tfstr = tfidf.transform(test_str)
test_tfstr.shape
lsvm.predict(test_tfstr.toarray())[0]

'B-LOC'

In [61]:

from sklearn.metrics import  accuracy_score
from sklearn import metrics
print("accuracy:", metrics.accuracy_score(test_lbl, y_pred_lsvm))

accuracy: 0.9351439398976132


In [62]:
import pickle
# save the model to disk
modelfilename = 'C:\my lap\Graduation proj\ArabicNER\Predication/lsvm.sav'
pickle.dump(lsvm, open(modelfilename, 'wb'))

#save vectorizer
vectorizerfilename = 'C:\my lap\Graduation proj\ArabicNER\Predication/vectorizer.pickle'
pickle.dump(vectorizer, open(vectorizerfilename, 'wb'))

#save tfidf
tfidffilename = 'C:\my lap\Graduation proj\ArabicNER\Predication/tfidf.pickle'
pickle.dump(tfidf, open(tfidffilename, 'wb'))

In [63]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
import pickle
import pandas as pd
# load the model from disk
modelfilename='C:\my lap\Graduation proj\ArabicNER\Predication/lsvm.sav'
loaded_model = pickle.load(open(modelfilename, 'rb'))

# load the vectorizer from disk
vectorizerfilename='C:\my lap\Graduation proj\ArabicNER\Predication/vectorizer.pickle'
loaded_vectorizer = pickle.load(open(vectorizerfilename, 'rb'))


# load the tfidf from disk
tfidffilename='C:\my lap\Graduation proj\ArabicNER\Predication/tfidf.pickle'
loaded_tfidf = pickle.load(open(tfidffilename, 'rb'))


In [64]:
phrase="هو مؤسس التقويم الهجري، وفي عهده بلغ الإسلام مبلغًا عظيمًا، وتوسع نطاق الدولة الإسلامية حتى شمل كامل العراق ومصر وليبيا والشام وفارس وخراسان وشرق الأناضول وجنوب أرمينية و سجستان،"

In [65]:
import sys
arr=phrase.split()
y=[]
token=[]
for x in arr:
    x=[x]
    test_str = loaded_vectorizer.transform(x)
    test_tfstr = loaded_tfidf.transform(test_str)
    test_tfstr.shape
    token.append(x)
    y.append(loaded_model.predict(test_tfstr.toarray())[0])

df=pd.DataFrame(list(zip(token,y)),columns=['token','entity_type'])
df

,token,entity_type
0,[هو],O
1,[مؤسس],O
2,[التقويم],O
3,[الهجري،],O
4,[وفي],O
5,[عهده],O
6,[بلغ],O
7,[الإسلام],B-MISC
8,[مبلغًا],O
9,[عظيمًا،],O
